### Data Ingestion Pipeline

In [1]:
import os
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from pathlib import Path

c:\Users\arany\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def process_excel(excel_directory, target_column=None):
    all_documents = []
    excel_dir = Path(excel_directory)
    excel_files = list(excel_dir.glob('**/*.xlsx'))

    if not excel_files:
        print(f"No Excel files found in {excel_directory}")
        return []

    for excel_file in excel_files:
        print(f"Processing: {excel_file.name}")
        try:
            df = pd.read_excel(excel_file)
            
            # Logic 1: If user provided a specific column AND it exists in this file
            if target_column and target_column in df.columns:
                loader_column = target_column
            
            # Logic 2: If no column provided (or it's missing), create a smart combined row
            else:
                loader_column = 'combined_text'
                # This attaches the header name to the value so the AI has perfect context
                df[loader_column] = df.apply(
                    lambda row: ' | '.join(f"{col}: {val}" for col, val in row.items() if pd.notnull(val)), 
                    axis=1
                )

            # Pass the dynamically selected column to the loader
            loader = DataFrameLoader(df, page_content_column=loader_column)
            documents = loader.load()

            for doc in documents:
                doc.metadata['source_file'] = excel_file.name
                doc.metadata['file_type'] = "xlsx"
                
            all_documents.extend(documents)
            
        except Exception as e:
            print(f"Error processing {excel_file.name}: {e}")

    print(f"Successfully loaded {len(all_documents)} dynamic chunks from Excel files.")
    return all_documents

# Example usage:
excel_docs = process_excel('../data/excel_files', target_column='YourColumn')
excel_docs[0:5]

Processing: New CSV for Morning Call.xlsx
Processing: New Updated CSV3.xlsx
Successfully loaded 92157 dynamic chunks from Excel files.


[Document(metadata={'Order Creation Date': Timestamp('2024-12-31 00:00:00'), 'OrderBusinessUnitName': 'The Morning Call', 'TotalPrice': 180.0, 'IsBillComplete': 0, 'source_file': 'New CSV for Morning Call.xlsx', 'file_type': 'xlsx'}, page_content='Order Creation Date: 2024-12-31 00:00:00 | OrderBusinessUnitName: The Morning Call | TotalPrice: 180.0 | IsBillComplete: 0'),
 Document(metadata={'Order Creation Date': Timestamp('2024-12-31 00:00:00'), 'OrderBusinessUnitName': 'The Morning Call', 'TotalPrice': 0.0, 'IsBillComplete': 1, 'source_file': 'New CSV for Morning Call.xlsx', 'file_type': 'xlsx'}, page_content='Order Creation Date: 2024-12-31 00:00:00 | OrderBusinessUnitName: The Morning Call | TotalPrice: 0.0 | IsBillComplete: 1'),
 Document(metadata={'Order Creation Date': Timestamp('2024-12-31 00:00:00'), 'OrderBusinessUnitName': 'The Morning Call', 'TotalPrice': 111.15, 'IsBillComplete': 0, 'source_file': 'New CSV for Morning Call.xlsx', 'file_type': 'xlsx'}, page_content='Order C

### Embedding and VectorStore DB

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
from typing import List
from sentence_transformers import SentenceTransformer

class EmbeddingManager:
    def __init__(self, model_name = 'all-MiniLM-L6-v2'):
        self.model_name = model_name
        self.model = None
        self._load_model(model_name)

    def _load_model(self, model_name):
        try:
            self.model = SentenceTransformer(self.model_name)
        except Exception as e:
            print(f"Error loading model '{model_name}': {e}")
            raise

    def generate_embeddings(self, texts: List[str]):
        if not self.model:
            raise ValueError("Model not loaded.")
        print(f"Generating embeddings for {len(texts)} texts...")
        try:
            embeddings = self.model.encode(texts, show_progress_bar=True)
            return embeddings
        except Exception as e:
            print(f"Error generating embeddings: {e}")
            raise

    def get_embedding_dimensions(self):
        if not self.model:
            raise ValueError("Model not loaded.")
        return self.model.get_sentence_embedding_dimension()
    
# Initialize the manager (Notice we removed the trailing variable!)
embedding_manager = EmbeddingManager()
print("✅ Embedding Manager successfully loaded!")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 109.94it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Embedding Manager successfully loaded!


### VectorStore

In [6]:
class VectorStore:
    def __init__(self, collection_name='all_excel_documents', persist_directory='../data/vector_store/excel/'):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_chromadb()

    def _initialize_chromadb(self):
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
        
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                # Updated this description to reflect your Excel data!
                metadata={"description": "Excel document chunks with embeddings"} 
            )

            print(f'Vector store initialized with collection: {self.collection_name}')
            
        except Exception as e:
            print(f"Error initializing ChromaDB: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents and embeddings must match.")
        
        # Prepare the chromadb entries
        ids = []
        metadata_list = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate a unique ID for each document
            doc_id = f'doc_{uuid.uuid4().hex[:8]}_{i}'
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadata_list.append(metadata)
              
            documents_text.append(doc.page_content)
            
            # Pro-tip: Ensure the math vector is a standard Python list for ChromaDB
            if isinstance(embedding, np.ndarray):
                embeddings_list.append(embedding.tolist())
            else:
                embeddings_list.append(embedding)

        try:
            self.collection.add(
                ids=ids,
                documents=documents_text,
                embeddings=embeddings_list,
                metadatas=metadata_list
            )

            print(f'Successfully added {len(documents)} documents to the vector store.')
        except Exception as e:
            print(f"Error adding documents to ChromaDB: {e}")
            raise

# Initialize the connection (Trailing variable removed!)
vector_store = VectorStore()
print("✅ Vector Store ready!")

Vector store initialized with collection: all_excel_documents
✅ Vector Store ready!


In [11]:
print("Cleaning 92,157 metadata dictionaries...")

# Connect our loaded Pandas documents
chunks = excel_docs

for chunk in chunks:
    # We iterate through the keys as a list so we can modify the dictionary safely
    for key in list(chunk.metadata.keys()):
        value = chunk.metadata[key]
        # If ChromaDB won't accept the type, we cast it to a standard string
        if not isinstance(value, (str, int, float, bool)):
            chunk.metadata[key] = str(value)
            
print("✅ Metadata cleaned and ready for ChromaDB!")

chunks[0:5]

Cleaning 92,157 metadata dictionaries...
✅ Metadata cleaned and ready for ChromaDB!


[Document(metadata={'Order Creation Date': '2024-12-31 00:00:00', 'OrderBusinessUnitName': 'The Morning Call', 'TotalPrice': 180.0, 'IsBillComplete': 0, 'source_file': 'New CSV for Morning Call.xlsx', 'file_type': 'xlsx'}, page_content='Order Creation Date: 2024-12-31 00:00:00 | OrderBusinessUnitName: The Morning Call | TotalPrice: 180.0 | IsBillComplete: 0'),
 Document(metadata={'Order Creation Date': '2024-12-31 00:00:00', 'OrderBusinessUnitName': 'The Morning Call', 'TotalPrice': 0.0, 'IsBillComplete': 1, 'source_file': 'New CSV for Morning Call.xlsx', 'file_type': 'xlsx'}, page_content='Order Creation Date: 2024-12-31 00:00:00 | OrderBusinessUnitName: The Morning Call | TotalPrice: 0.0 | IsBillComplete: 1'),
 Document(metadata={'Order Creation Date': '2024-12-31 00:00:00', 'OrderBusinessUnitName': 'The Morning Call', 'TotalPrice': 111.15, 'IsBillComplete': 0, 'source_file': 'New CSV for Morning Call.xlsx', 'file_type': 'xlsx'}, page_content='Order Creation Date: 2024-12-31 00:00:00

In [13]:
from tqdm.auto import tqdm

print(f"Preparing to safely embed {len(chunks)} rows into ChromaDB...")

# Processing 100 rows at a time
batch_size = 5000

try:
    for i in tqdm(range(0, len(chunks), batch_size), desc="Embedding Progress"):
        # Grab the current slice
        batch_chunks = chunks[i : i + batch_size]
        
        # Extract the text string
        texts_to_embed = [chunk.page_content for chunk in batch_chunks]
        
        # Generate the math vectors
        new_embeddings = embedding_manager.generate_embeddings(texts_to_embed)
        
        # Add them directly to your existing ChromaDB collection
        vector_store.add_documents(batch_chunks, new_embeddings)
        
    print("\n✅ Success! All 92,157 rows of Morning Call data are completely ingested!")
    
except Exception as e:
    print(f"\n❌ Error adding to database: {e}")

Preparing to safely embed 92157 rows into ChromaDB...


Embedding Progress:   0%|          | 0/19 [00:00<?, ?it/s]

Generating embeddings for 5000 texts...


Embedding Progress:   5%|▌         | 1/19 [02:13<39:59, 133.32s/it]

Successfully added 5000 documents to the vector store.
Generating embeddings for 5000 texts...


...

Successfully added 5000 documents to the vector store.
Generating embeddings for 5000 texts...


Embedding Progress:  95%|█████████▍| 18/19 [38:20<02:08, 128.32s/it]

Successfully added 5000 documents to the vector store.
Generating embeddings for 2157 texts...


Embedding Progress: 100%|██████████| 19/19 [39:19<00:00, 124.19s/it]

Successfully added 2157 documents to the vector store.

✅ Success! All 92,157 rows of Morning Call data are completely ingested!


### Retriever pipeline from vector store

In [29]:
class RAGRetriever:
    """Handles the retrieval of relevant document chunks based on a query."""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 50, score_threshold: float = 0.5):
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        try:
            # Perform the query to get relevant documents
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k,
                include=['documents', 'metadatas', 'embeddings', 'distances']
            )

            retrieved_docs = []
            documents = results.get('documents', [])[0]
            metadatas = results.get('metadatas', [])[0]
            distances = results.get('distances', [])[0]
            ids = results.get('ids', [])[0]

            # Check if the query returned any results
            if not documents:
                print("No documents found")
                return []

            # Process the results and filter based on similarity score
            for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                similarity_score = 1.0 - (distance / 2.0) # Higher similarity means a smaller distance
                
                if similarity_score >= score_threshold:
                    retrieved_docs.append({
                        'id': doc_id,
                        'content': document,
                        'metadata': metadata,
                        'similarity_score': similarity_score,
                        'distance': distance,
                        'rank': i + 1
                    })

            if not retrieved_docs:
                print(f"No documents meet the score threshold of {score_threshold}")
            
            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            raise


# Make sure vector_store and embedding_manager are instantiated before passing them to the RAGRetriever
vector_store = VectorStore()  # Assuming you have this class set up
embedding_manager = EmbeddingManager()  # Assuming you have this class set up


Vector store initialized with collection: all_excel_documents


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 112.92it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [46]:
# 1. Initialize the retriever (using your existing connections)
rag_retriever = RAGRetriever(vector_store, embedding_manager)

# 2. Write a test query that matches your Excel data
query = "What is the TotalPrice for The Morning Call?"

# 3. Run the search! (Notice we increased top_k and lowered the threshold)
print(f"Searching 92,000+ rows for: '{query}'\n")
retrieved_docs = rag_retriever.retrieve(query, top_k=10, score_threshold=0.2)

# 4. Print out the results so we can see what the AI will read
if retrieved_docs:
    print(f"✅ Found {len(retrieved_docs)} matching rows!\n")
    print("--- TOP 3 MATCHES ---")
    for i in range(min(3, len(retrieved_docs))):
        doc = retrieved_docs[i]
        print(f"Match {i+1} (Score: {doc['similarity_score']:.2f}):")
        print(doc['content'])
        print("-" * 40)
else:
    print("❌ No matching rows found. Try lowering the threshold further.")

Searching 92,000+ rows for: 'What is the TotalPrice for The Morning Call?'

Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 18.29it/s]

✅ Found 10 matching rows!

--- TOP 3 MATCHES ---
Match 1 (Score: 0.55):
Order Creation Date: 2022-10-10 00:00:00 | OrderBusinessUnitName: The Morning Call | TotalPrice: 86.4025 | IsBillComplete: 1
----------------------------------------
Match 2 (Score: 0.54):
Order Creation Date: 2024-10-10 00:00:00 | OrderBusinessUnitName: The Morning Call | TotalPrice: 88.275 | IsBillComplete: 1
----------------------------------------
Match 3 (Score: 0.54):
Order Creation Date: 2024-10-10 00:00:00 | OrderBusinessUnitName: The Morning Call | TotalPrice: 88.275 | IsBillComplete: 1
----------------------------------------


In [39]:
from google import genai
import os
from dotenv import load_dotenv

load_dotenv() 

api_key = os.getenv("GOOGLE_API_KEY")

class simple_rag_retriever:
    def __init__(self, api_key):
        self.client = genai.Client(api_key=api_key)
        self.model_name = "gemini-2.5-flash-lite" 
        
    def generate_response(self, query, retrieved_docs):
        if not retrieved_docs:
            return "Error: The retriever returned 0 chunks. The context is empty!"

        # Context Injection
        context_chunks = [doc['content'] for doc in retrieved_docs]
        context_string = "\n\n---\n\n".join(context_chunks)

        # Constructing the prompt
        prompt = f"""You are a helpful assistant answering questions based on the provided documents.
Using only the text below, answer the question. If the answer is not contained in the context, say "I do not have enough information to answer that."
        
Context:
{context_string}

Question: {query}

Answer:"""
        
        try:
            # THE FIX: Explicitly stating model= and contents=
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=prompt
            )
            return response.text.strip()
        except Exception as e:
            print(f"Error generating response: {e}")
            raise

# 1. Ask a specific question about your columns!
query = "get me OrderBusinessUnitName names"

# 2. Retrieve the documents (Bumping up top_k so Gemini gets more context!)
print("🔍 Searching the database...")
retrieved_docs = rag_retriever.retrieve(query, top_k=50, score_threshold=0.0)

# 3. Initialize generator
generator = simple_rag_retriever(api_key)

# 4. Generate the final answer
print("🧠 Gemini is reading the rows and generating an answer...")
final_answer = generator.generate_response(query, retrieved_docs)

print("\n" + "="*25)
print("=== FINAL RAG ANSWER ===")
print("="*25)
print(final_answer)

🔍 Searching the database...
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.51it/s]


🧠 Gemini is reading the rows and generating an answer...
Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 45.363378829s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTi

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 45.363378829s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-fla

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
from pathlib import Path

# 1. Ensure your data is loaded into a single DataFrame (You likely already have this!)
excel_files = list(Path('../data/excel_files').glob('**/*.xlsx'))
all_data = pd.concat([pd.read_excel(f) for f in excel_files])

# 2. Connect to Gemini (Using the LangChain integration)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    google_api_key=api_key,
    temperature=0 # Keep it at 0 so it writes strict, accurate Pandas code
)

# 3. Create the Agent
pandas_agent = create_pandas_dataframe_agent(
    llm,
    all_data, 
    verbose=True, # Set to True so you can watch the AI write the Python code!
    allow_dangerous_code=True # Required to allow the AI to run Pandas functions locally
)

In [45]:
import time

def smart_ask(query):
    print("🔍 STEP 1: Asking the Vector Database...")
    
    # 1. Try the Vector RAG first
    retrieved_docs = rag_retriever.retrieve(query, top_k=20, score_threshold=0.2)
    rag_answer = generator.generate_response(query, retrieved_docs)
    
    # 2. Check if the Vector Database failed!
    if "I do not have enough information" in rag_answer:
        print("⚠️ Vector DB doesn't have the global view. Switching to Pandas Agent...")
        print("🤖 STEP 2: Agent is analyzing the entire DataFrame...")
        
        # 3. Trigger the Pandas Agent as a fallback
        try:
            pandas_response = pandas_agent.invoke(query)
            
            print("\n" + "="*30)
            print("=== FINAL ANSWER (From Pandas) ===")
            print("="*30)
            print(pandas_response['output'])
            
        except Exception as e:
            print(f"\n❌ Both methods failed. Agent Error: {e}")
            
    else:
        # If it didn't say the failure phrase, the RAG succeeded!
        print("\n" + "="*30)
        print("=== FINAL ANSWER (From Vector DB) ===")
        print("="*30)
        print(rag_answer)

# --- Test it out! ---
print("--- TEST 1: A qualitative question ---")
smart_ask("What is the TotalPrice for the orders from The Morning Call? Tell me if the bills are complete.")

time.sleep(25)

print("\n\n--- TEST 2: A global aggregation question ---")
smart_ask("What are all the unique OrderBusinessUnitName values in the dataset?")

--- TEST 1: A qualitative question ---
🔍 STEP 1: Asking the Vector Database...
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.21it/s]


Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 7.007285062s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'g

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 7.007285062s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flas